**Script de recherche et d'appariement des données Geod'Air**

**Description**

- Récupération automatique des données des 10 stations les plus proches, quelle que soit la typologie de la station (de fond, de proximité, etc.), sur la période d’enquête d’intérêt pour le **NO2, PM2.5, PM10 et benzène (C6H6)**.

- Calcul des distances des stations aux bâtiments

- Qualification des données récupérées (indicateur(s) de qualité de la mesure, données manquantes, identification des valeurs aberrantes (ex : concentration < 0) )

- Construction d’une table structurée : identifiant du bâtiment, période d’enquête, longitude et latitude du bâtiment, identifiants des stations récupérées, longitudes et latitudes des stations, distances entre le bâtiment et les stations, typologies des stations (de fond, de proximité), caractère mobile ou fixe des stations, substances et concentrations mesurées, période de mesure, indicateur(s) de qualité des mesures, limites de détection et quantification.

# Imports

In [1]:
# Automatic reload of the Jupyter Notebook
%load_ext autoreload
%autoreload 2

# Load the .env (for the API key)
%load_ext dotenv

In [2]:
# Imports
import os
import requests
import datetime
import numpy as np
import pandas as pd
import tempfile
import io
from math import radians, cos, sin, sqrt, atan2

# Géod'Air - Chargement API Stations

Cette API Geod'Air permet d'obtenir la liste des stations de mesure ainsi que toutes les informations les caractérisant (type de site, code, adresse, date de mise en service, etc.).

In [3]:
# Chargement de la clé API Geod'Air
apikey_geodair = os.environ.get('apikey_geodair')

if apikey_geodair is not None:
    print('✅ apikey_geodair variable loaded')

else:
    print('❌ apikey_geodair variable not loaded')

✅ apikey_geodair variable loaded


In [4]:
# API GET request pour obtenir les données concernant les stations Geod'air

# Paramètres à changer si besoin
date=datetime.date.today()
apikey=apikey_geodair

url_stations = f'https://www.geodair.fr/api-ext/station/export?date={date}'
headers = {'accept': 'text/csv', 'apikey': apikey}

req = requests.get(url_stations, headers=headers)

In [5]:
# API Response headers
req.headers

{'Date': 'Fri, 26 May 2023 10:18:38 GMT', 'Content-Type': 'text/csv;charset=ISO-8859-1', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'RateLimit-Remaining': '13', 'RateLimit-Reset': '2483', 'X-RateLimit-Remaining-Hour': '13', 'X-RateLimit-Limit-Hour': '15', 'RateLimit-Limit': '15', 'X-Token-Expires-In': '2488.9493250847', 'Vary': 'Origin, Access-Control-Request-Method, Access-Control-Request-Headers, Origin', 'Content-Disposition': 'attachment; filename="Stations_2023-05-26.csv"', 'X-XSS-Protection': '1; mode=block', 'Cache-Control': 'no-cache, no-store, max-age=0, must-revalidate', 'Pragma': 'no-cache', 'Expires': '0', 'Access-Control-Allow-Origin': '*', 'Access-Control-Expose-Headers': 'X-Auth-Token', 'X-Kong-Upstream-Latency': '309', 'X-Kong-Proxy-Latency': '2', 'Via': 'kong/2.6.0'}

In [6]:
# API data raw
data_stations = req.content.decode('ISO-8859-1')

In [7]:
# API data into a Pandas DataFrame
df_stations = pd.read_csv(io.StringIO(data_stations), sep=";")

In [8]:
# Ne garder que les informations essentielles concernant les stations Geod'air
df_stations_geodair = df_stations[[
    "Type de site",
    "Code",
    "Alias",
    "Longitude",
    "Latitude",
    "Implantation",
    "Mobilité"]]

df_stations_geodair

,Type de site,Code,Alias,Longitude,Latitude,Implantation,Mobilité
0,classique,FR28028,Bethune Stade,2.635692,50.535385,Urbaine,Station Fixe
1,classique,FR38024,RDT,55.302844,-21.213212,Rurale près des villes,Station Fixe
2,classique,FR07056,NaN,4.226006,44.983770,Rurale régionale,Station Fixe
3,classique,FR02043,Marignane,5.222281,43.416531,Urbaine,Station Fixe
4,classique,FR34043,Oysonville,1.955611,48.391139,Rurale régionale,Station Fixe
...,...,...,...,...,...,...,...
559,classique,FR12047,BESSIERES ECONOTRE,1.595475,43.803139,Rurale près des villes,Station Fixe
560,classique,FR22009,Pays Naborien (Carling),6.716886,49.165726,Périurbaine,Station Fixe
561,classique,FR08018,Montpellier St Gely,3.800210,43.692600,Périurbaine,Station Fixe
562,classique,FR05074,Le Havre Ec. Herriot,0.100703,49.489630,Urbaine,Station Fixe


# CSTB - Bâtiments

Chargement de la liste des bâtiments CSTB pour lesquels l'appariement aux données Géod'Air doit être fait.

In [9]:
# Obtenir les données concernant les bâtiments CSTB
cstb_file = '../../data/cstb/API2BUILD_COORD_DATES_BATI.xlsx'
df_immeubles_cstb = pd.read_excel(cstb_file)

In [10]:
df_immeubles_cstb

,Identifiant,Longitude,Latitude,Date_debut_enquete,Date_fin_enquete
0,B001,5.365996,43.314652,2016-11-29,NaT
1,B002,5.389293,43.310217,2016-11-25,NaT
2,B003,5.369872,43.310963,2016-12-16,NaT
3,B004,5.379358,43.286702,2017-06-12,NaT
4,B005,5.041559,43.417900,2017-06-06,NaT
...,...,...,...,...,...
1101,L568,2.273457,48.825051,2022-11-08,2022-11-15
1102,L569,2.847896,49.990562,2022-09-02,2022-09-09
1103,L570,2.158504,48.940902,2022-09-01,2022-09-08
1104,L571,1.879346,48.757418,2022-04-04,2022-04-11


# Obtention données polluant

Les données ont été stockées en local sur le path **/Volumes/NO NAME/API2Build/data/geodair/'**, puis dans un sous-dossier pour chaque polluant (NO2, PM2.5, PM10 & C6H6).

**Pour obtenir les dataframes clean, changer ce path pour le faire correspondre à l'emplacement où se trouvent ces fichiers de polluants en local.**

### NO2

In [11]:
# Sélectionner où se trouvent les fichiers NO2
no2_path = '/Volumes/NO NAME/API2Build/data/geodair/NO2'

In [12]:
%%time
# Créer 1 DataFrame par fichier
df1 = pd.read_csv(f'{no2_path}/Export Moy. journalière - NO₂ - 2013-06-15 00_00 - 2014-01-01 00_00.csv', sep=";")
df2 = pd.read_csv(f'{no2_path}/Export Moy. journalière - NO₂ - 2014-01-01 00_00 - 2014-07-01 00_00.csv', sep=";")
df3 = pd.read_csv(f'{no2_path}/Export Moy. journalière - NO₂ - 2014-07-01 00_00 - 2015-01-01 00_00.csv', sep=";")
df4 = pd.read_csv(f'{no2_path}/Export Moy. journalière - NO₂ - 2015-01-01 00_00 - 2015-07-01 00_00.csv', sep=";")
df5 = pd.read_csv(f'{no2_path}/Export Moy. journalière - NO₂ - 2015-07-01 00_00 - 2016-01-01 00_00.csv', sep=";")
df6 = pd.read_csv(f'{no2_path}/Export Moy. journalière - NO₂ - 2016-01-01 00_00 - 2016-07-01 00_00.csv', sep=";")
df7 = pd.read_csv(f'{no2_path}/Export Moy. journalière - NO₂ - 2016-07-01 00_00 - 2017-01-01 00_00.csv', sep=";")
df8 = pd.read_csv(f'{no2_path}/Export Moy. journalière - NO₂ - 2017-01-01 00_00 - 2017-07-01 00_00.csv', sep=";")
df9 = pd.read_csv(f'{no2_path}/Export Moy. journalière - NO₂ - 2017-07-01 00_00 - 2018-01-01 00_00.csv', sep=";")
df10 = pd.read_csv(f'{no2_path}/Export Moy. journalière - NO₂ - 2018-01-01 00_00 - 2018-07-01 00_00.csv', sep=";")
df11 = pd.read_csv(f'{no2_path}/Export Moy. journalière - NO₂ - 2018-07-01 00_00 - 2019-01-01 00_00.csv', sep=";")
df12 = pd.read_csv(f'{no2_path}/Export Moy. journalière - NO₂ - 2019-01-01 00_00 - 2019-07-01 00_00.csv', sep=";")
df13 = pd.read_csv(f'{no2_path}/Export Moy. journalière - NO₂ - 2019-07-01 00_00 - 2020-01-01 00_00.csv', sep=";")
df14 = pd.read_csv(f'{no2_path}/Export Moy. journalière - NO₂ - 2020-01-01 00_00 - 2020-07-01 00_00.csv', sep=";")
df15 = pd.read_csv(f'{no2_path}/Export Moy. journalière - NO₂ - 2020-07-01 00_00 - 2021-01-01 00_00.csv', sep=";")
df16 = pd.read_csv(f'{no2_path}/Export Moy. journalière - NO₂ - 2021-01-01 00_00 - 2021-07-01 00_00.csv', sep=";")
df17 = pd.read_csv(f'{no2_path}/Export Moy. journalière - NO₂ - 2021-07-01 00_00 - 2022-01-01 00_00.csv', sep=";")
df18 = pd.read_csv(f'{no2_path}/Export Moy. journalière - NO₂ - 2022-01-01 00_00 - 2022-07-01 00_00.csv', sep=";")
df19 = pd.read_csv(f'{no2_path}/Export Moy. journalière - NO₂ - 2022-07-01 00_00 - 2023-01-01 00_00.csv', sep=";")
df20 = pd.read_csv(f'{no2_path}/Export Moy. journalière - NO₂ - 2023-01-01 00_00 - 2023-03-01 00_00.csv', sep=";")

CPU times: user 9.07 s, sys: 1.63 s, total: 10.7 s
Wall time: 12.4 s


In [13]:
%%time
# Concaténer tous les DataFrames en 1 seul DataFrame
no2_data_full = pd.concat([df1,
                      df2,
                      df3,
                      df4,
                      df5,
                      df6,
                      df7,
                      df8,
                      df9,
                      df10,
                      df11,
                      df12,
                      df13,
                      df14,
                      df15,
                      df16,
                      df17,
                      df18,
                      df19,
                      df20], axis=0)

CPU times: user 646 ms, sys: 281 ms, total: 927 ms
Wall time: 988 ms


In [14]:
no2_data_full.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1446534 entries, 0 to 22607
Data columns (total 23 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   Date de début          1446534 non-null  object 
 1   Date de fin            1446534 non-null  object 
 2   Organisme              1446534 non-null  object 
 3   code zas               1446534 non-null  object 
 4   Zas                    1446534 non-null  object 
 5   code site              1446534 non-null  object 
 6   nom site               1446534 non-null  object 
 7   type d'implantation    1446534 non-null  object 
 8   Polluant               1446534 non-null  object 
 9   type d'influence       1446534 non-null  object 
 10  Réglementaire          1446534 non-null  object 
 11  type d'évaluation      1446534 non-null  object 
 12  type de valeur         1446534 non-null  object 
 13  valeur                 1446534 non-null  float64
 14  valeur brute           14

In [15]:
# Sélectionner les informations essentielles du polluant
no2_data = no2_data_full[[
    "Date de début",
    "Date de fin",
    "Organisme",
    "code site",
    "nom site",
    "type d'implantation",
    "Polluant",
    "type d'influence",
    "type d'évaluation",
    "valeur brute",
    "unité de mesure",
    "code qualité"]]

In [16]:
no2_data

,Date de début,Date de fin,Organisme,code site,nom site,type d'implantation,Polluant,type d'influence,type d'évaluation,valeur brute,unité de mesure,code qualité
0,2013/06/15 00:00:00,2013/06/15 23:59:59,ATMO GRAND EST,FR01001,Longlaville-Ecole,Périurbaine,NO2,Fond,mesures fixes,4.725000,µg-m3,A
1,2013/06/15 00:00:00,2013/06/15 23:59:59,ATMO GRAND EST,FR01004,Distroff,Rurale régionale,NO2,Fond,mesures fixes,0.000000,µg-m3,N
2,2013/06/15 00:00:00,2013/06/15 23:59:59,ATMO GRAND EST,FR01005,Hayange,Périurbaine,NO2,Industrielle,mesures fixes,9.862500,µg-m3,R
3,2013/06/15 00:00:00,2013/06/15 23:59:59,ATMO GRAND EST,FR01011,Metz-Centre,Urbaine,NO2,Fond,mesures fixes,12.745833,µg-m3,A
4,2013/06/15 00:00:00,2013/06/15 23:59:59,ATMO GRAND EST,FR01012,Metz-Borny,Urbaine,NO2,Fond,mesures fixes,11.137500,µg-m3,R
...,...,...,...,...,...,...,...,...,...,...,...,...
22603,2023/02/28 00:00:00,2023/02/28 23:59:59,ATMO BOURGOGNE-FRANCHE-COMTE,FR82040,Dole centre,Urbaine,NO2,Fond,mesures fixes,11.025000,µg-m3,A
22604,2023/02/28 00:00:00,2023/02/28 23:59:59,ATMO BOURGOGNE-FRANCHE-COMTE,FR82041,Tavaux,Rurale près des villes,NO2,Industrielle,mesures fixes,8.504167,µg-m3,A
22605,2023/02/28 00:00:00,2023/02/28 23:59:59,ATMO BOURGOGNE-FRANCHE-COMTE,FR82042,Damparis,Rurale près des villes,NO2,Industrielle,mesures fixes,9.341667,µg-m3,R
22606,2023/02/28 00:00:00,2023/02/28 23:59:59,ATMO BOURGOGNE-FRANCHE-COMTE,FR82060,Vesoul Pres Caillet,Urbaine,NO2,Fond,mesures fixes,15.404167,µg-m3,A


In [17]:
# Convertir les dates en datetime
no2_data['Date de début'] = pd.to_datetime(no2_data['Date de début'])
no2_data['Date de fin'] = pd.to_datetime(no2_data['Date de fin'])

/var/folders/1j/7bkgcqrn02n90wb3kn4sf4ww0000gn/T/ipykernel_95198/1395938147.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no2_data['Date de début'] = pd.to_datetime(no2_data['Date de début'])
/var/folders/1j/7bkgcqrn02n90wb3kn4sf4ww0000gn/T/ipykernel_95198/1395938147.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no2_data['Date de fin'] = pd.to_datetime(no2_data['Date de fin'])


In [18]:
no2_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1446534 entries, 0 to 22607
Data columns (total 12 columns):
 #   Column               Non-Null Count    Dtype         
---  ------               --------------    -----         
 0   Date de début        1446534 non-null  datetime64[ns]
 1   Date de fin          1446534 non-null  datetime64[ns]
 2   Organisme            1446534 non-null  object        
 3   code site            1446534 non-null  object        
 4   nom site             1446534 non-null  object        
 5   type d'implantation  1446534 non-null  object        
 6   Polluant             1446534 non-null  object        
 7   type d'influence     1446534 non-null  object        
 8   type d'évaluation    1446534 non-null  object        
 9   valeur brute         1446534 non-null  float64       
 10  unité de mesure      1446534 non-null  object        
 11  code qualité         1446534 non-null  object        
dtypes: datetime64[ns](2), float64(1), object(9)
memory usage: 143.5

### PM2.5

In [19]:
# Sélectionner où se trouvent les fichiers PM25
pm25_path = '/Volumes/NO NAME/API2Build/data/geodair/PM25'

In [20]:
%%time
# Créer 1 DataFrame par fichier
df1 = pd.read_csv(f'{pm25_path}/Export Moy. journalière - PM₂.₅ - 2013-06-15 00_00 - 2014-01-01 00_00.csv', sep=";")
df2 = pd.read_csv(f'{pm25_path}/Export Moy. journalière - PM₂.₅ - 2014-01-01 00_00 - 2014-07-01 00_00.csv', sep=";")
df3 = pd.read_csv(f'{pm25_path}/Export Moy. journalière - PM₂.₅ - 2014-07-01 00_00 - 2015-01-01 00_00.csv', sep=";")
df4 = pd.read_csv(f'{pm25_path}/Export Moy. journalière - PM₂.₅ - 2015-01-01 00_00 - 2015-07-01 00_00.csv', sep=";")
df5 = pd.read_csv(f'{pm25_path}/Export Moy. journalière - PM₂.₅ - 2015-07-01 00_00 - 2016-01-01 00_00.csv', sep=";")
df6 = pd.read_csv(f'{pm25_path}/Export Moy. journalière - PM₂.₅ - 2016-01-01 00_00 - 2016-07-01 00_00.csv', sep=";")
df7 = pd.read_csv(f'{pm25_path}/Export Moy. journalière - PM₂.₅ - 2016-07-01 00_00 - 2017-01-01 00_00.csv', sep=";")
df8 = pd.read_csv(f'{pm25_path}/Export Moy. journalière - PM₂.₅ - 2017-01-01 00_00 - 2017-07-01 00_00.csv', sep=";")
df9 = pd.read_csv(f'{pm25_path}/Export Moy. journalière - PM₂.₅ - 2017-07-01 00_00 - 2018-01-01 00_00.csv', sep=";")
df10 = pd.read_csv(f'{pm25_path}/Export Moy. journalière - PM₂.₅ - 2018-01-01 00_00 - 2018-07-01 00_00.csv', sep=";")
df11 = pd.read_csv(f'{pm25_path}/Export Moy. journalière - PM₂.₅ - 2018-07-01 00_00 - 2019-01-01 00_00.csv', sep=";")
df12 = pd.read_csv(f'{pm25_path}/Export Moy. journalière - PM₂.₅ - 2019-01-01 00_00 - 2019-07-01 00_00.csv', sep=";")
df13 = pd.read_csv(f'{pm25_path}/Export Moy. journalière - PM₂.₅ - 2019-07-01 00_00 - 2020-01-01 00_00.csv', sep=";")
df14 = pd.read_csv(f'{pm25_path}/Export Moy. journalière - PM₂.₅ - 2020-01-01 00_00 - 2020-07-01 00_00.csv', sep=";")
df15 = pd.read_csv(f'{pm25_path}/Export Moy. journalière - PM₂.₅ - 2020-07-01 00_00 - 2021-01-01 00_00.csv', sep=";")
df16 = pd.read_csv(f'{pm25_path}/Export Moy. journalière - PM₂.₅ - 2021-01-01 00_00 - 2021-07-01 00_00.csv', sep=";")
df17 = pd.read_csv(f'{pm25_path}/Export Moy. journalière - PM₂.₅ - 2021-07-01 00_00 - 2022-01-01 00_00.csv', sep=";")
df18 = pd.read_csv(f'{pm25_path}/Export Moy. journalière - PM₂.₅ - 2022-01-01 00_00 - 2022-07-01 00_00.csv', sep=";")
df19 = pd.read_csv(f'{pm25_path}/Export Moy. journalière - PM₂.₅ - 2022-07-01 00_00 - 2023-01-01 00_00.csv', sep=";")
df20 = pd.read_csv(f'{pm25_path}/Export Moy. journalière - PM₂.₅ - 2023-01-01 00_00 - 2023-03-01 00_00.csv', sep=";")

CPU times: user 4.44 s, sys: 948 ms, total: 5.38 s
Wall time: 6.53 s


In [21]:
%%time
# Concaténer tous les DataFrames en 1 seul DataFrame
pm25_data_full = pd.concat([df1,
                      df2,
                      df3,
                      df4,
                      df5,
                      df6,
                      df7,
                      df8,
                      df9,
                      df10,
                      df11,
                      df12,
                      df13,
                      df14,
                      df15,
                      df16,
                      df17,
                      df18,
                      df19,
                      df20], axis=0)

CPU times: user 273 ms, sys: 122 ms, total: 395 ms
Wall time: 441 ms


In [22]:
# Sélectionner les informations essentielles du polluant
pm25_data = pm25_data_full[[
    "Date de début",
    "Date de fin",
    "Organisme",
    "code site",
    "nom site",
    "type d'implantation",
    "Polluant",
    "type d'influence",
    "type d'évaluation",
    "valeur brute",
    "unité de mesure",
    "code qualité"]]

In [23]:
pm25_data

,Date de début,Date de fin,Organisme,code site,nom site,type d'implantation,Polluant,type d'influence,type d'évaluation,valeur brute,unité de mesure,code qualité
0,2013/06/15 00:00:00,2013/06/15 23:59:59,ATMO GRAND EST,FR01011,Metz-Centre,Urbaine,PM2.5,Fond,mesures fixes,8.887500,µg-m3,A
1,2013/06/15 00:00:00,2013/06/15 23:59:59,ATMO GRAND EST,FR01020,Thionville-Centre,Urbaine,PM2.5,Fond,mesures fixes,7.887500,µg-m3,A
2,2013/06/15 00:00:00,2013/06/15 23:59:59,ATMO SUD,FR02020,Rognac les Brets,Périurbaine,PM2.5,Fond,mesures fixes,15.178261,µg-m3,R
3,2013/06/15 00:00:00,2013/06/15 23:59:59,ATMO SUD,FR03006,MARSEILLE RABATAU,Urbaine,PM2.5,Trafic,mesures fixes,16.100000,µg-m3,R
4,2013/06/15 00:00:00,2013/06/15 23:59:59,ATMO SUD,FR03029,AIX CENTRE ECOLE ART,Urbaine,PM2.5,Fond,mesures fixes,14.810000,µg-m3,R
...,...,...,...,...,...,...,...,...,...,...,...,...
13116,2023/02/28 00:00:00,2023/02/28 23:59:59,ATMO BOURGOGNE-FRANCHE-COMTE,FR82006,Besancon Prevoyance,Urbaine,PM2.5,Fond,mesures fixes,12.541667,µg-m3,A
13117,2023/02/28 00:00:00,2023/02/28 23:59:59,ATMO BOURGOGNE-FRANCHE-COMTE,FR82010,Montbéliard centre,Urbaine,PM2.5,Fond,mesures fixes,10.837500,µg-m3,A
13118,2023/02/28 00:00:00,2023/02/28 23:59:59,ATMO BOURGOGNE-FRANCHE-COMTE,FR82050,Lons-le-Saunier CV,Urbaine,PM2.5,Fond,mesures fixes,13.225000,µg-m3,A
13119,2023/02/28 00:00:00,2023/02/28 23:59:59,ATMO BOURGOGNE-FRANCHE-COMTE,FR82060,Vesoul Pres Caillet,Urbaine,PM2.5,Fond,mesures fixes,13.362500,µg-m3,A


In [24]:
# Convertir les dates en datetime
pm25_data['Date de début'] = pd.to_datetime(pm25_data['Date de début'])
pm25_data['Date de fin'] = pd.to_datetime(pm25_data['Date de fin'])

/var/folders/1j/7bkgcqrn02n90wb3kn4sf4ww0000gn/T/ipykernel_95198/2540570352.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pm25_data['Date de début'] = pd.to_datetime(pm25_data['Date de début'])
/var/folders/1j/7bkgcqrn02n90wb3kn4sf4ww0000gn/T/ipykernel_95198/2540570352.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pm25_data['Date de fin'] = pd.to_datetime(pm25_data['Date de fin'])


In [25]:
pm25_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 568412 entries, 0 to 13120
Data columns (total 12 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   Date de début        568412 non-null  datetime64[ns]
 1   Date de fin          568412 non-null  datetime64[ns]
 2   Organisme            568412 non-null  object        
 3   code site            568412 non-null  object        
 4   nom site             568412 non-null  object        
 5   type d'implantation  568412 non-null  object        
 6   Polluant             568412 non-null  object        
 7   type d'influence     568412 non-null  object        
 8   type d'évaluation    568412 non-null  object        
 9   valeur brute         568412 non-null  float64       
 10  unité de mesure      568412 non-null  object        
 11  code qualité         568412 non-null  object        
dtypes: datetime64[ns](2), float64(1), object(9)
memory usage: 56.4+ MB


### PM10

In [26]:
# Sélectionner où se trouvent les fichiers PM10
pm10_path = '/Volumes/NO NAME/API2Build/data/geodair/PM10'

In [27]:
%%time
# Créer 1 DataFrame par fichier
df1 = pd.read_csv(f'{pm10_path}/Export Moy. journalière - PM₁₀ - 2013-06-15 00_00 - 2014-01-01 00_00.csv', sep=";")
df2 = pd.read_csv(f'{pm10_path}/Export Moy. journalière - PM₁₀ - 2014-01-01 00_00 - 2014-07-01 00_00.csv', sep=";")
df3 = pd.read_csv(f'{pm10_path}/Export Moy. journalière - PM₁₀ - 2014-07-01 00_00 - 2015-01-01 00_00.csv', sep=";")
df4 = pd.read_csv(f'{pm10_path}/Export Moy. journalière - PM₁₀ - 2015-01-01 00_00 - 2015-07-01 00_00.csv', sep=";")
df5 = pd.read_csv(f'{pm10_path}/Export Moy. journalière - PM₁₀ - 2015-07-01 00_00 - 2016-01-01 00_00.csv', sep=";")
df6 = pd.read_csv(f'{pm10_path}/Export Moy. journalière - PM₁₀ - 2016-01-01 00_00 - 2016-07-01 00_00.csv', sep=";")
df7 = pd.read_csv(f'{pm10_path}/Export Moy. journalière - PM₁₀ - 2016-07-01 00_00 - 2017-01-01 00_00.csv', sep=";")
df8 = pd.read_csv(f'{pm10_path}/Export Moy. journalière - PM₁₀ - 2017-01-01 00_00 - 2017-07-01 00_00.csv', sep=";")
df9 = pd.read_csv(f'{pm10_path}/Export Moy. journalière - PM₁₀ - 2017-07-01 00_00 - 2018-01-01 00_00.csv', sep=";")
df10 = pd.read_csv(f'{pm10_path}/Export Moy. journalière - PM₁₀ - 2018-01-01 00_00 - 2018-07-01 00_00.csv', sep=";")
df11 = pd.read_csv(f'{pm10_path}/Export Moy. journalière - PM₁₀ - 2018-07-01 00_00 - 2019-01-01 00_00.csv', sep=";")
df12 = pd.read_csv(f'{pm10_path}/Export Moy. journalière - PM₁₀ - 2019-01-01 00_00 - 2019-07-01 00_00.csv', sep=";")
df13 = pd.read_csv(f'{pm10_path}/Export Moy. journalière - PM₁₀ - 2019-07-01 00_00 - 2020-01-01 00_00.csv', sep=";")
df14 = pd.read_csv(f'{pm10_path}/Export Moy. journalière - PM₁₀ - 2020-01-01 00_00 - 2020-07-01 00_00.csv', sep=";")
df15 = pd.read_csv(f'{pm10_path}/Export Moy. journalière - PM₁₀ - 2020-07-01 00_00 - 2021-01-01 00_00.csv', sep=";")
df16 = pd.read_csv(f'{pm10_path}/Export Moy. journalière - PM₁₀ - 2021-01-01 00_00 - 2021-07-01 00_00.csv', sep=";")
df17 = pd.read_csv(f'{pm10_path}/Export Moy. journalière - PM₁₀ - 2021-07-01 00_00 - 2022-01-01 00_00.csv', sep=";")
df18 = pd.read_csv(f'{pm10_path}/Export Moy. journalière - PM₁₀ - 2022-01-01 00_00 - 2022-07-01 00_00.csv', sep=";")
df19 = pd.read_csv(f'{pm10_path}/Export Moy. journalière - PM₁₀ - 2022-07-01 00_00 - 2023-01-01 00_00.csv', sep=";")
df20 = pd.read_csv(f'{pm10_path}/Export Moy. journalière - PM₁₀ - 2023-01-01 00_00 - 2023-03-01 00_00.csv', sep=";")

CPU times: user 9.28 s, sys: 1.72 s, total: 11 s
Wall time: 12.1 s


In [28]:
%%time
# Concaténer tous les DataFrames en 1 seul DataFrame
pm10_data_full = pd.concat([df1,
                      df2,
                      df3,
                      df4,
                      df5,
                      df6,
                      df7,
                      df8,
                      df9,
                      df10,
                      df11,
                      df12,
                      df13,
                      df14,
                      df15,
                      df16,
                      df17,
                      df18,
                      df19,
                      df20], axis=0)

CPU times: user 558 ms, sys: 230 ms, total: 788 ms
Wall time: 800 ms


In [29]:
# Sélectionner les informations essentielles du polluant
pm10_data = pm10_data_full[[
    "Date de début",
    "Date de fin",
    "Organisme",
    "code site",
    "nom site",
    "type d'implantation",
    "Polluant",
    "type d'influence",
    "type d'évaluation",
    "valeur brute",
    "unité de mesure",
    "code qualité"]]

In [30]:
pm10_data

,Date de début,Date de fin,Organisme,code site,nom site,type d'implantation,Polluant,type d'influence,type d'évaluation,valeur brute,unité de mesure,code qualité
0,2013/06/15 00:00:00,2013/06/15 23:59:59,ATMO GRAND EST,FR01001,Longlaville-Ecole,Périurbaine,PM10,Fond,mesures fixes,7.537500,µg-m3,A
1,2013/06/15 00:00:00,2013/06/15 23:59:59,ATMO GRAND EST,FR01005,Hayange,Périurbaine,PM10,Industrielle,mesures fixes,9.962500,µg-m3,A
2,2013/06/15 00:00:00,2013/06/15 23:59:59,ATMO GRAND EST,FR01006,Florange,Périurbaine,PM10,Fond,mesures fixes,0.000000,µg-m3,N
3,2013/06/15 00:00:00,2013/06/15 23:59:59,ATMO GRAND EST,FR01009,Gandrange,Périurbaine,PM10,Industrielle,mesures fixes,10.554167,µg-m3,A
4,2013/06/15 00:00:00,2013/06/15 23:59:59,ATMO GRAND EST,FR01011,Metz-Centre,Urbaine,PM10,Fond,mesures fixes,9.387500,µg-m3,A
...,...,...,...,...,...,...,...,...,...,...,...,...
20318,2023/02/28 00:00:00,2023/02/28 23:59:59,ATMO BOURGOGNE-FRANCHE-COMTE,FR82030,Montandon Baresans,Rurale nationale,PM10,Fond,mesures fixes,13.825000,µg-m3,A
20319,2023/02/28 00:00:00,2023/02/28 23:59:59,ATMO BOURGOGNE-FRANCHE-COMTE,FR82040,Dole centre,Urbaine,PM10,Fond,mesures fixes,13.783333,µg-m3,A
20320,2023/02/28 00:00:00,2023/02/28 23:59:59,ATMO BOURGOGNE-FRANCHE-COMTE,FR82043,Chatenois,Périurbaine,PM10,Industrielle,mesures fixes,12.458333,µg-m3,A
20321,2023/02/28 00:00:00,2023/02/28 23:59:59,ATMO BOURGOGNE-FRANCHE-COMTE,FR82060,Vesoul Pres Caillet,Urbaine,PM10,Fond,mesures fixes,21.333333,µg-m3,A


In [31]:
# Convertir les dates en datetime
pm10_data['Date de début'] = pd.to_datetime(pm10_data['Date de début'])
pm10_data['Date de fin'] = pd.to_datetime(pm10_data['Date de fin'])

/var/folders/1j/7bkgcqrn02n90wb3kn4sf4ww0000gn/T/ipykernel_95198/420219753.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pm10_data['Date de début'] = pd.to_datetime(pm10_data['Date de début'])
/var/folders/1j/7bkgcqrn02n90wb3kn4sf4ww0000gn/T/ipykernel_95198/420219753.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pm10_data['Date de fin'] = pd.to_datetime(pm10_data['Date de fin'])


In [32]:
pm10_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1303076 entries, 0 to 20322
Data columns (total 12 columns):
 #   Column               Non-Null Count    Dtype         
---  ------               --------------    -----         
 0   Date de début        1303076 non-null  datetime64[ns]
 1   Date de fin          1303076 non-null  datetime64[ns]
 2   Organisme            1303076 non-null  object        
 3   code site            1303076 non-null  object        
 4   nom site             1303076 non-null  object        
 5   type d'implantation  1303076 non-null  object        
 6   Polluant             1303076 non-null  object        
 7   type d'influence     1303076 non-null  object        
 8   type d'évaluation    1303076 non-null  object        
 9   valeur brute         1303076 non-null  float64       
 10  unité de mesure      1303076 non-null  object        
 11  code qualité         1303076 non-null  object        
dtypes: datetime64[ns](2), float64(1), object(9)
memory usage: 129.2

### C6H6

In [33]:
# Sélectionner où se trouvent le fichier C6H6
c6h6_path = '/Volumes/NO NAME/API2Build/data/geodair/C6H6'

In [34]:
# Créer 1 DataFrame par fichier
c6h6_data_full = pd.read_csv(f'{c6h6_path}/Export Moy. journalière - C₆H₆ - 2013-06-15 00_00 - 2023-03-01 00_00.csv', sep=";")

In [35]:
c6h6_data_full.head()

,Date de début,Date de fin,Organisme,code zas,Zas,code site,nom site,type d'implantation,Polluant,type d'influence,...,valeur,valeur brute,unité de mesure,taux de saisie,couverture temporelle,couverture de données,code qualité,validité,Latitude,Longitude
0,2013/06/15 00:00:00,2013/06/15 23:59:59,ATMO SUD,FR02N30,PROVENCE-ALPES-COTE-D-AZUR-ZI,FR02001,Berre l'Etang,Périurbaine,C6H6,Industrielle,...,0.56,0.564348,µg-m3,96.0,100,96.0,A,1,43.486234,5.171939
1,2013/06/15 00:00:00,2013/06/15 23:59:59,ATMO SUD,FR02N30,PROVENCE-ALPES-COTE-D-AZUR-ZI,FR02028,Martigues Lavera,Périurbaine,C6H6,Industrielle,...,0.50,0.503750,µg-m3,100.0,100,100.0,A,1,43.386564,5.026868
2,2013/06/15 00:00:00,2013/06/15 23:59:59,ATMO AUVERGNE-RHÔNE-ALPES,FR07A01,AUVERGNE-CLERMONT,FR07034,Clermont-Esplanade Gare,Urbaine,C6H6,Trafic,...,0.71,0.711667,µg-m3,100.0,100,100.0,A,1,45.775696,3.096250
3,2013/06/15 00:00:00,2013/06/15 23:59:59,ATMO HAUTS DE FRANCE,FR11N10,NORD-PAS-DE-CALAIS-ZUR,FR10012,Mardyck,Périurbaine,C6H6,Industrielle,...,0.18,0.175000,µg-m3,100.0,100,100.0,A,1,51.019430,2.250990
4,2013/06/15 00:00:00,2013/06/15 23:59:59,ATMO HAUTS DE FRANCE,FR11A01,NORD-PAS-DE-CALAIS-LILLE,FR11034,Roubaix Serres,Urbaine,C6H6,Trafic,...,0.18,0.181250,µg-m3,100.0,100,100.0,A,1,50.706535,3.180660


In [36]:
# Sélectionner les informations essentielles du polluant
c6h6_data = c6h6_data_full[[
    "Date de début",
    "Date de fin",
    "Organisme",
    "code site",
    "nom site",
    "type d'implantation",
    "Polluant",
    "type d'influence",
    "type d'évaluation",
    "valeur brute",
    "unité de mesure",
    "code qualité"]]

In [37]:
c6h6_data

,Date de début,Date de fin,Organisme,code site,nom site,type d'implantation,Polluant,type d'influence,type d'évaluation,valeur brute,unité de mesure,code qualité
0,2013/06/15 00:00:00,2013/06/15 23:59:59,ATMO SUD,FR02001,Berre l'Etang,Périurbaine,C6H6,Industrielle,mesures fixes,0.564348,µg-m3,A
1,2013/06/15 00:00:00,2013/06/15 23:59:59,ATMO SUD,FR02028,Martigues Lavera,Périurbaine,C6H6,Industrielle,mesures fixes,0.503750,µg-m3,A
2,2013/06/15 00:00:00,2013/06/15 23:59:59,ATMO AUVERGNE-RHÔNE-ALPES,FR07034,Clermont-Esplanade Gare,Urbaine,C6H6,Trafic,mesures fixes,0.711667,µg-m3,A
3,2013/06/15 00:00:00,2013/06/15 23:59:59,ATMO HAUTS DE FRANCE,FR10012,Mardyck,Périurbaine,C6H6,Industrielle,mesures fixes,0.175000,µg-m3,A
4,2013/06/15 00:00:00,2013/06/15 23:59:59,ATMO HAUTS DE FRANCE,FR11034,Roubaix Serres,Urbaine,C6H6,Trafic,mesures fixes,0.181250,µg-m3,A
...,...,...,...,...,...,...,...,...,...,...,...,...
38658,2023/02/28 00:00:00,2023/02/28 23:59:59,ATMO HAUTS DE FRANCE,FR10012,Mardyck,Périurbaine,C6H6,Industrielle,estimation objective,1.354167,µg-m3,A
38659,2023/02/28 00:00:00,2023/02/28 23:59:59,ATMO HAUTS DE FRANCE,FR11007,Lille Leeds,Urbaine,C6H6,Trafic,estimation objective,0.727500,µg-m3,A
38660,2023/02/28 00:00:00,2023/02/28 23:59:59,ATMO AUVERGNE-RHÔNE-ALPES,FR20029,FEYZIN STADE,Périurbaine,C6H6,Industrielle,estimation objective,2.200417,µg-m3,A
38661,2023/02/28 00:00:00,2023/02/28 23:59:59,ATMO AUVERGNE-RHÔNE-ALPES,FR20064,VERNAISON,Périurbaine,C6H6,Industrielle,estimation objective,0.832500,µg-m3,A


In [38]:
# Convertir les dates en datetime
c6h6_data['Date de début'] = pd.to_datetime(c6h6_data['Date de début'])
c6h6_data['Date de fin'] = pd.to_datetime(c6h6_data['Date de fin'])

/var/folders/1j/7bkgcqrn02n90wb3kn4sf4ww0000gn/T/ipykernel_95198/2902365235.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  c6h6_data['Date de début'] = pd.to_datetime(c6h6_data['Date de début'])
/var/folders/1j/7bkgcqrn02n90wb3kn4sf4ww0000gn/T/ipykernel_95198/2902365235.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  c6h6_data['Date de fin'] = pd.to_datetime(c6h6_data['Date de fin'])


In [39]:
c6h6_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38663 entries, 0 to 38662
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Date de début        38663 non-null  datetime64[ns]
 1   Date de fin          38663 non-null  datetime64[ns]
 2   Organisme            38663 non-null  object        
 3   code site            38663 non-null  object        
 4   nom site             38663 non-null  object        
 5   type d'implantation  38663 non-null  object        
 6   Polluant             38663 non-null  object        
 7   type d'influence     38663 non-null  object        
 8   type d'évaluation    38663 non-null  object        
 9   valeur brute         38663 non-null  float64       
 10  unité de mesure      38663 non-null  object        
 11  code qualité         38663 non-null  object        
dtypes: datetime64[ns](2), float64(1), object(9)
memory usage: 3.5+ MB


# Script appariement CSTB-Geod'Air

## Calcul 10 stations Geod'Air les plus proches

In [40]:
%%time
# Créer la fonction pour calculer la distance haversine entre 2 points
def haversine(lat1, lon1, lat2, lon2):
    """
    Calculer la distance haversine entre 2 points en kilomètres
    """
    
    R = 6371  # rayon de la Terre en km
    phi1 = radians(lat1)
    phi2 = radians(lat2)
    delta_phi = radians(lat2 - lat1)
    delta_lambda = radians(lon2 - lon1)
    
    a = sin(delta_phi / 2) ** 2 + cos(phi1) * cos(phi2) * sin(delta_lambda / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    d = R * c
    
    return d

# Calculer la distance haversine de chaque immeuble CSTB avec toutes les stations Geod'air
distances_df = pd.DataFrame()

for _, immeuble in df_immeubles_cstb.iterrows():
    distances = []
    
    for _, station in df_stations_geodair.iterrows():
        distance = haversine(immeuble['Latitude'], immeuble['Longitude'],
                             station['Latitude'], station['Longitude'])
        distances.append(distance)
        
    distances_df[immeuble['Identifiant']] = distances


<timed exec>:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.ins

<timed exec>:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.ins

<timed exec>:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.ins

<timed exec>:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.ins

<timed exec>:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.ins

...

<timed exec>:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.ins

<timed exec>:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.ins

<timed exec>:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.ins

CPU times: user 2min 2s, sys: 3.4 s, total: 2min 6s
Wall time: 2min 40s


<timed exec>:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`


In [41]:
distances_df

,B001,B002,B003,B004,B005,B006,B007,B008,B009,B010,...,L563,L564,L565,L566,L567,L568,L569,L570,L571,L572
0,829.093880,830.014747,829.566041,832.368348,812.084963,562.435227,560.994611,562.887761,440.412484,440.051074,...,196.023879,190.869341,185.833766,196.023879,62.431075,191.957305,62.431075,180.583252,205.060609,180.583252
1,8800.361714,8798.653598,8799.851916,8797.440350,8827.323985,9395.393012,9395.850725,9394.790756,9067.977144,9068.291468,...,9396.616127,9396.357034,9396.512456,9396.616127,9445.605209,9390.151597,9445.605209,9404.983847,9406.836362,9404.983847
2,206.680379,207.947588,207.186138,209.950058,185.855433,438.329133,438.201732,437.916649,286.704654,287.025283,...,453.658455,456.294726,459.203701,453.658455,566.250593,452.113604,566.250593,467.101501,455.897395,467.101501
3,16.226448,17.945259,16.737670,19.227852,14.597412,587.579643,587.795908,587.046523,430.538640,430.922866,...,644.648991,647.128000,649.876793,644.648991,752.981576,642.781356,752.981576,657.871668,647.259949,657.871668
4,623.055469,624.270703,623.557577,626.335229,602.216842,342.457166,341.106144,342.772003,327.633547,327.423363,...,47.551348,53.086159,58.587102,47.551348,189.296044,53.610057,189.296044,62.920434,41.113098,62.920434
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
559,308.613813,310.559124,309.002744,310.302809,280.710643,338.739970,339.447390,338.083312,514.305849,514.557723,...,557.510595,562.312624,567.118127,557.510595,694.531631,560.826971,694.531631,572.919180,551.321378,572.919180
560,658.821857,659.030579,659.180802,661.735096,651.916627,677.913190,676.753546,678.051756,219.557376,219.184928,...,337.319201,332.426947,328.102469,337.319201,293.604164,326.337540,293.604164,333.068344,356.016295,333.068344
561,133.092108,135.035100,133.522272,135.148834,104.591403,476.033187,476.359162,475.467325,431.118577,431.458370,...,582.657313,586.038942,589.603775,582.657313,704.020743,582.613881,704.020743,597.008527,582.192099,597.008527
562,796.065152,797.400897,796.575481,799.311920,773.601447,383.237508,381.792133,383.810259,501.849710,501.596130,...,165.930128,167.714170,169.211736,165.930128,205.108231,174.420233,205.108231,161.434980,152.902954,161.434980


In [42]:
%%time
# Pour chaque bâtiment CSTB, obtenir les 10 stations Geod'air les plus proches
# Stations Geod'air classées de la plus proche à la plus lointaine
closest10_stations_df_code = pd.DataFrame()
closest10_stations_df_km = pd.DataFrame()

for immeuble_cstb in df_immeubles_cstb['Identifiant']:
    distances = distances_df[immeuble_cstb].sort_values()
    
    closest10_stations_code = []
    closest10_stations_km = []
    
    for i in range(10):
        geodair_station_id = df_stations_geodair.loc[distances.index[i], 'Code']
        closest10_stations_code.append(geodair_station_id)
        
        closest10_stations_km.append(distances.iloc[i])
        
    closest10_stations_df_code[immeuble_cstb] = closest10_stations_code
    closest10_stations_df_km[immeuble_cstb] = closest10_stations_km

<timed exec>:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.ins

<timed exec>:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.ins

<timed exec>:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.ins

<timed exec>:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.ins

<timed exec>:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.ins

...

<timed exec>:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.ins

<timed exec>:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.ins

<timed exec>:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.ins

CPU times: user 4.86 s, sys: 342 ms, total: 5.2 s
Wall time: 8.3 s


<timed exec>:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.ins

In [43]:
closest10_stations_df_code

,B001,B002,B003,B004,B005,B006,B007,B008,B009,B010,...,L563,L564,L565,L566,L567,L568,L569,L570,L571,L572
0,FR03043,FR03043,FR03043,FR03006,FR02004,FR09003,FR09003,FR09003,FR82006,FR82006,...,FR04029,FR04179,FR04150,FR04029,FR18045,FR04179,FR18045,FR04051,FR04181,FR04051
1,FR03014,FR03006,FR03014,FR03043,FR02028,FR09008,FR09008,FR09008,FR82001,FR82001,...,FR04179,FR04029,FR04017,FR04179,FR18042,FR04060,FR18042,FR04150,FR04029,FR04150
2,FR03006,FR03014,FR03006,FR03014,FR02008,FR09203,FR09203,FR09203,FR82005,FR82005,...,FR04150,FR04150,FR04060,FR04150,FR18080,FR04012,FR18080,FR04002,FR04038,FR04002
3,FR03037,FR03037,FR03037,FR03037,FR02029,FR09303,FR09303,FR09303,FR26114,FR26114,...,FR04060,FR04017,FR04031,FR04060,FR28022,FR04031,FR28022,FR04017,FR04049,FR04017
4,FR02043,FR03032,FR03032,FR03032,FR02007,FR09305,FR09305,FR09305,FR82070,FR82070,...,FR04017,FR04060,FR04179,FR04017,FR18053,FR04071,FR18053,FR04060,FR04331,FR04060
5,FR03032,FR03030,FR02043,FR02043,FR02006,FR09302,FR09302,FR09302,FR82043,FR82043,...,FR04049,FR04031,FR04002,FR04049,FR18079,FR04017,FR18079,FR04031,FR04063,FR04031
6,FR03030,FR02043,FR03030,FR03030,FR02011,FR23152,FR23152,FR23152,FR82040,FR82040,...,FR04012,FR04012,FR04131,FR04012,FR18035,FR04037,FR18035,FR04023,FR04179,FR04023
7,FR02021,FR02021,FR02021,FR02021,FR02013,FR23124,FR23124,FR23124,FR82060,FR82060,...,FR04031,FR04071,FR04118,FR04031,FR06003,FR04118,FR06003,FR04058,FR04150,FR04058
8,FR02029,FR03021,FR02029,FR02029,FR02012,FR23078,FR23078,FR23078,FR82042,FR82042,...,FR04071,FR04131,FR04051,FR04071,FR28115,FR04055,FR28115,FR04131,FR04180,FR04131
9,FR03021,FR02029,FR03021,FR03021,FR02021,FR09304,FR09304,FR09304,FR82041,FR82041,...,FR04118,FR04118,FR04071,FR04118,FR28010,FR04131,FR28010,FR04179,FR04060,FR04179


In [44]:
closest10_stations_df_km

,B001,B002,B003,B004,B005,B006,B007,B008,B009,B010,...,L563,L564,L565,L566,L567,L568,L569,L570,L571,L572
0,2.546476,0.702204,2.106949,1.850120,0.167111,1.657120,0.490297,2.308166,1.053717,1.251146,...,0.562673,6.361774,0.848142,0.562673,32.847896,0.648300,32.847896,4.895682,14.212567,4.895682
1,3.743729,3.810966,4.198500,2.411541,3.680957,3.241721,3.669285,2.777591,2.232074,2.551101,...,10.901557,6.377889,3.026038,10.901557,34.849155,3.858521,34.849155,8.140158,18.983689,8.140158
2,4.948423,4.823265,4.435722,6.996668,5.133624,4.595279,4.835714,5.078185,5.874987,6.047747,...,12.839069,6.659928,5.099630,12.839069,35.120188,3.967788,35.120188,10.027464,19.514071,10.027464
3,12.270026,10.322771,11.857035,10.693344,6.137501,55.135066,54.192815,55.133365,12.805967,13.082888,...,13.577066,7.967987,5.745414,13.577066,35.510676,5.629741,35.510676,10.960105,22.716682,10.960105
4,16.226448,15.081891,16.651681,15.785088,6.500747,56.472010,55.506786,56.484484,29.263873,29.160425,...,14.058418,8.037556,6.447562,14.058418,40.112510,5.710609,40.112510,13.554059,26.966459,13.554059
5,17.015497,17.124324,16.737670,19.227852,6.713874,58.473970,57.989157,58.224378,35.543099,35.621556,...,14.090248,9.673027,6.598578,14.090248,42.384058,6.273663,42.384058,13.780094,29.527949,13.780094
6,17.459932,17.945259,17.683286,19.853263,8.098220,60.999505,59.908775,61.696561,41.813416,41.927320,...,14.948381,10.553089,6.934686,14.948381,45.580568,6.367932,45.580568,14.440741,29.736303,14.440741
7,20.170523,22.095424,20.524455,21.743375,9.768832,64.793648,63.355284,65.243311,42.827367,42.460768,...,15.383270,11.058415,7.160506,15.383270,45.814048,6.629655,45.814048,14.579381,30.350503,14.579381
8,22.511793,24.250644,22.970194,24.912898,10.895315,102.898546,101.461230,103.487120,48.853265,48.971821,...,16.360349,11.099774,7.539168,16.360349,48.715222,6.641291,48.715222,14.630834,31.374725,14.630834
9,24.213915,24.431673,24.526220,26.973590,11.432972,108.366830,107.053567,108.643488,50.938925,51.071327,...,16.791689,11.160772,7.830653,16.791689,51.126074,6.790390,51.126074,14.742451,32.281365,14.742451


## Appariement CSTB-Geod'Air

In [ ]:
%%time
# Sélection du bâtiment CSTB à étudier
identifiant_immeuble_cstb = 'L568' # Mettre identifiant bâtiment CSTB

# Sélection des polluants à ajouter dans la table
pollutants = ['NO2', 'PM2.5', 'PM10', 'C6H6']

# Sélection des dates de la période d'enquête pour le bâtiment CSTB sélectionné
row = df_immeubles_cstb[df_immeubles_cstb['Identifiant']==identifiant_immeuble_cstb]

start_date = row['Date_debut_enquete'].values[0]
end_date = row['Date_fin_enquete'].values[0]

# Si date_fin_enquete = 'NaT', cela signifie que la période d'enquête était de 1 jour
if pd.isnull(end_date):
    end_date = start_date

# Création de la plage de dates de la période d'enquête pour le bâtiment CSTB sélectionné
dates = pd.date_range(start_date,end_date,freq='D')

# Création de la table pour le bâtiment CSTB sélectionnné
cstb_top10stationsGeodair = pd.DataFrame(columns=[
    'CSTB_Code',
    'CSTB_Longitude',
    'CSTB_Latitude',
    'CSTB_Date_debut_enquete',
    'CSTB_Date_fin_enquete',    
    'Geodair_Code',
    'Geodair_Longitude',
    'Geodair_Latitude',
    'Geodair_DistanceToCSTBBuilding',
    'Geodair_Implantation',
    'Geodair_Mobilité',
    'Geodair_Polluant',
    'Geodair_Date',
    'Geodair_Valeur_brute',
    'Geodair_Unité_mesure',
    'Geodair_Code_qualité'
    ])

for geodair_station_id in closest10_stations_df_code[identifiant_immeuble_cstb]:
    
    for pollutant in pollutants:
        
        for date in dates:
    
            # Création liste vide pour chaque station Geod'air
            geodair_station_list = []
            
            # Ajout informations sur le bâtiment CSTB
            geodair_station_list.append(identifiant_immeuble_cstb)
            geodair_station_list.append(df_immeubles_cstb[df_immeubles_cstb['Identifiant']==identifiant_immeuble_cstb]['Longitude'].iloc[0])
            geodair_station_list.append(df_immeubles_cstb[df_immeubles_cstb['Identifiant']==identifiant_immeuble_cstb]['Latitude'].iloc[0])
            geodair_station_list.append(df_immeubles_cstb[df_immeubles_cstb['Identifiant']==identifiant_immeuble_cstb]['Date_debut_enquete'].iloc[0])
            geodair_station_list.append(df_immeubles_cstb[df_immeubles_cstb['Identifiant']==identifiant_immeuble_cstb]['Date_fin_enquete'].iloc[0])

            # Ajout informations sur la station Geod'air
            geodair_station_list.append(geodair_station_id)
            geodair_station_list.append(df_stations_geodair[df_stations_geodair['Code']==geodair_station_id]['Longitude'].iloc[0])
            geodair_station_list.append(df_stations_geodair[df_stations_geodair['Code']==geodair_station_id]['Latitude'].iloc[0])

            # Ajout de la distance du bâtiment CSTB à la station Geod'air sélectionnée
            index_station = closest10_stations_df_code.index[closest10_stations_df_code[identifiant_immeuble_cstb]==geodair_station_id][0]
            geodair_station_list.append(closest10_stations_df_km[identifiant_immeuble_cstb].iloc[index_station])

            # Ajout informations complémentaires sur la station Geod'air
            geodair_station_list.append(df_stations_geodair[df_stations_geodair['Code']==geodair_station_id]['Implantation'].iloc[0])
            geodair_station_list.append(df_stations_geodair[df_stations_geodair['Code']==geodair_station_id]['Mobilité'].iloc[0])

            # Ajout information sur le polluant sélectionné
            geodair_station_list.append(pollutant)
            
            # Ajout date sélectionnée dans la période d'enquête pour le bâtiment CSTB
            geodair_station_list.append(date)
            
            # Ajout informations sur le polluant sélectionné à la date sélectionnée
            if pollutant == 'NO2':            
                if len(no2_data[(no2_data['Date de début']==date) & (no2_data['code site']==geodair_station_id)])==0:
                    geodair_station_list.append(np.nan)
                    geodair_station_list.append(np.nan)
                    geodair_station_list.append(np.nan)
                else:
                    geodair_station_list.append(no2_data[(no2_data['Date de début']==date) & (no2_data['code site']==geodair_station_id)]['valeur brute'].values[0])
                    geodair_station_list.append(no2_data[(no2_data['Date de début']==date) & (no2_data['code site']==geodair_station_id)]['unité de mesure'].values[0])
                    geodair_station_list.append(no2_data[(no2_data['Date de début']==date) & (no2_data['code site']==geodair_station_id)]['code qualité'].values[0])

            elif pollutant == 'PM2.5':            
                if len(pm25_data[(pm25_data['Date de début']==date) & (pm25_data['code site']==geodair_station_id)])==0:
                    geodair_station_list.append(np.nan)
                    geodair_station_list.append(np.nan)
                    geodair_station_list.append(np.nan)
                else:
                    geodair_station_list.append(pm25_data[(pm25_data['Date de début']==date) & (pm25_data['code site']==geodair_station_id)]['valeur brute'].values[0])
                    geodair_station_list.append(pm25_data[(pm25_data['Date de début']==date) & (pm25_data['code site']==geodair_station_id)]['unité de mesure'].values[0])
                    geodair_station_list.append(pm25_data[(pm25_data['Date de début']==date) & (pm25_data['code site']==geodair_station_id)]['code qualité'].values[0])
                    
            elif pollutant == 'PM10':            
                if len(pm10_data[(pm10_data['Date de début']==date) & (pm10_data['code site']==geodair_station_id)])==0:
                    geodair_station_list.append(np.nan)
                    geodair_station_list.append(np.nan)
                    geodair_station_list.append(np.nan)
                else:
                    geodair_station_list.append(pm10_data[(pm10_data['Date de début']==date) & (pm10_data['code site']==geodair_station_id)]['valeur brute'].values[0])
                    geodair_station_list.append(pm10_data[(pm10_data['Date de début']==date) & (pm10_data['code site']==geodair_station_id)]['unité de mesure'].values[0])
                    geodair_station_list.append(pm10_data[(pm10_data['Date de début']==date) & (pm10_data['code site']==geodair_station_id)]['code qualité'].values[0])

            elif pollutant == 'C6H6':            
                if len(c6h6_data[(c6h6_data['Date de début']==date) & (c6h6_data['code site']==geodair_station_id)])==0:
                    geodair_station_list.append(np.nan)
                    geodair_station_list.append(np.nan)
                    geodair_station_list.append(np.nan)
                else:
                    geodair_station_list.append(c6h6_data[(c6h6_data['Date de début']==date) & (c6h6_data['code site']==geodair_station_id)]['valeur brute'].values[0])
                    geodair_station_list.append(c6h6_data[(c6h6_data['Date de début']==date) & (c6h6_data['code site']==geodair_station_id)]['unité de mesure'].values[0])
                    geodair_station_list.append(c6h6_data[(c6h6_data['Date de début']==date) & (c6h6_data['code site']==geodair_station_id)]['code qualité'].values[0])

            # Ajout de la liste complète au DataFrame dans une nouvelle ligne
            cstb_top10stationsGeodair.loc[len(cstb_top10stationsGeodair)] = geodair_station_list
        
cstb_top10stationsGeodair


## Export output appariement

In [ ]:
# Exporter le DataFrame si besoin
cstb_top10stationsGeodair.to_csv(f'{os.getcwd()[:40]}data/geodair/{identifiant_immeuble_cstb}_top10Geodair.csv')
print(f'Export available at {os.getcwd()[:40]}data/geodair/')